In [1]:
import pandas as pd
import os
import numpy as np
import pickle
import gc
from sklearn.preprocessing import LabelEncoder
import multiprocessing as mp

In [2]:
data_path = '../../../data'
features_path = '../1pre_features'
answer_info = pd.read_csv(os.path.join(data_path, 'answer_info_0926.txt'), header=None, sep='\t')
#train1 = pd.read_csv(os.path.join(data_path, 'invite_info_0926.txt'), header=None, sep='\t')

#train1.columns = ['问题id', '用户id', '邀请创建时间','是否回答']
answer_info.columns = ['回答id', '问题id', '用户id', '回答创建时间', '回答内容的单字编码序列', '回答内容的切词编码序列', '回答是否被标优', '回答是否被推荐', '回答是否被收入圆桌', '是否包含图片', '是否包含视频', '回答字数', '点赞数', '取赞数', '评论数', '收藏数', '感谢数', '举报数', '没有帮助数', '反对数']

drop_feat = ['回答内容的单字编码序列', '回答内容的切词编码序列', '回答是否被标优', '回答是否被推荐', '回答是否被收入圆桌', '是否包含图片', '是否包含视频',]
answer_info  = answer_info.drop(drop_feat, axis=1)

#user_answer_info = pd.merge(train1, answer_info, how='left', on='用户id')

In [3]:
answer_info['回答创建时间-hour'] = answer_info['回答创建时间'].apply(lambda x:x.split('-')[1].split('H')[1]).astype(int)
answer_info['回答创建时间-day'] = answer_info['回答创建时间'].apply(lambda x:x.split('-')[0].split('D')[1]).astype(int)

In [4]:
question_info = pd.read_csv(os.path.join(data_path, 'question_info_0926.txt'), header=None, sep='\t')
question_info.columns = ['问题id','问题创建时间','问题标题单字编码','问题标题切词编码','问题描述单字编码','问题描述切词编码','问题绑定话题']
drop_feat = ['问题创建时间','问题标题单字编码','问题标题切词编码','问题描述单字编码','问题描述切词编码',]
question_info  = question_info.drop(drop_feat, axis=1)

answer_qestion_info = pd.merge(answer_info, question_info, on='问题id', how='left')

### 生成问题的几个补充特征

In [39]:
question_info1 = pd.read_csv(os.path.join(data_path, 'question_info_0926.txt'), header=None, sep='\t')
question_info1.columns = ['问题id','问题创建时间','问题标题单字编码','问题标题切词编码','问题描述单字编码','问题描述切词编码','问题绑定话题']
drop_feat = ['问题标题单字编码','问题描述单字编码','问题绑定话题']
question_info1  = question_info1.drop(drop_feat, axis=1)
print(question_info1.info())

question_info1['q_day'] = question_info1['问题创建时间'].apply(lambda x:x.split('-')[0].split('D')[1]).astype(int)
question_info1['q_hour'] = question_info1['问题创建时间'].apply(lambda x:x.split('-')[1].split('H')[1]).astype(int)
question_info1['q_title_num'] = question_info1['问题标题切词编码'].apply(lambda x:len(x)).astype(int)
drop_feat = ['问题标题切词编码','问题描述切词编码', '问题创建时间']
question_info1  = question_info1.drop(drop_feat, axis=1)
print(question_info1.head(5))

question_info1.to_hdf('question_info1.hd5', key='data')

train_test_data = pd.read_hdf(os.path.join(features_path,'data0.h5'), key='data')
train_test_data = train_test_data[['qid', 'day', 'hour']]
train_test_data = pd.merge(train_test_data, question_info1, left_on='qid', right_on='问题id', how='left').drop('问题id', axis=1)
print(train_test_data.head(5))

train_test_data['diff_qi_days'] = train_test_data['day'] - train_test_data['q_day']
train_test_data['diff_qi_hours'] = train_test_data['hour'] - train_test_data['q_hour']
train_test_data.loc[:,'diff_qi_hours'] += 23
drop_feat = ['q_day','q_hour']
train_test_data  = train_test_data.drop(drop_feat, axis=1)
print(train_test_data.head(5))

print(train_test_data.info())

train_test_data.to_hdf('q_ex_features.h5', key='data')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3735352 entries, 0 to 3735351
Data columns (total 6 columns):
qid              object
day              int32
hour             int32
q_title_num      int64
diff_qi_days     int64
diff_qi_hours    int64
dtypes: int32(2), int64(3), object(1)
memory usage: 171.0+ MB
None


### ---分割线---

In [5]:
answer_qestion_info  = answer_qestion_info[['回答id','用户id','问题id','问题绑定话题','回答字数','点赞数','感谢数','收藏数','回答创建时间-day']]
print(answer_qestion_info.head())

          回答id         用户id         问题id                       问题绑定话题  回答字数  \
0  A2502060945   M625498202  Q1867533817  T381,T8211,T3144,T4936,T823    41   
1  A2847829478   M142330444  Q3366788616           T5490,T2180,T17098   204   
2  A2005999231   M771499642  Q4264694221           T8716,T10196,T8767    54   
3    A14821523  M2282072267  Q1088851650                         T258    42   
4   A731550034  M2282072267  Q1023877868                     T95,T545    44   

   点赞数  感谢数  收藏数  回答创建时间-day  
0    1    1    0        3808  
1    1    1    3        3810  
2    2    0    0        3853  
3    1    1    0        3859  
4    0    0    0        3855  


In [6]:
# 减少内存占用
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int' or str(col_type)[:5] == 'float':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            
            if str(col_type)[:5] == 'float':
                df[col] = df[col].astype(np.float32)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

def creat_2pkl(answer_qestion_info, train_label, etype):
    answer_qestion_info1 = answer_qestion_info['问题绑定话题'].str.split(',', expand=True).stack()
    #print(answer_qestion_info1)
    answer_qestion_info1 = answer_qestion_info1.reset_index(level=1, drop=True).rename('user_topic_id')
    answer_qestion_info = answer_qestion_info.join(answer_qestion_info1)
    print(answer_qestion_info.head(10))
    print(answer_qestion_info.info())
    del answer_qestion_info1
    answer_qestion_info = reduce_mem_usage(answer_qestion_info)
    
    user_topic_word = answer_qestion_info[['用户id','user_topic_id','回答字数']]
    t1 = answer_qestion_info.groupby(['用户id','user_topic_id'])['回答字数'].agg('sum').to_frame()
    t1.columns = ['u_topic_word_count']
    t1 = t1.reset_index()
    user_topic_word = pd.merge(user_topic_word, t1, on=['用户id','user_topic_id'], how='left')
    del user_topic_word['回答字数'], t1
    user_topic_word = user_topic_word.drop_duplicates(subset=None, keep='first', inplace=False)
    print(user_topic_word.info())
    user_topic_word = reduce_mem_usage(user_topic_word)
    
    label_question = train_label['qid'].to_frame()
    label_question = label_question.drop_duplicates(subset=None, keep='first', inplace=False)
    question_topic = pd.merge(label_question, answer_qestion_info, left_on='qid', right_on='问题id', how='left').drop(['qid'],axis=1)
    question_topic = question_info[['问题id','问题绑定话题']].drop_duplicates(subset=None, keep='first', inplace=False)
    #print(question_topic.info())
    
    question_topic1 = question_topic['问题绑定话题'].str.split(',', expand=True).stack()
    question_topic1 = question_topic1.reset_index(level=1, drop=True).rename('question_topic_id')
    question_topic = question_topic.join(question_topic1)
    del question_topic1, question_topic['问题绑定话题']
    #print(question_topic.head(10))
    print(question_topic.info())
    question_topic = reduce_mem_usage(question_topic)
    
    label_users = train_label['uid'].to_frame()
    label_users = label_users.drop_duplicates(subset=None, keep='first', inplace=False)
    user_topic_word = pd.merge(label_users, user_topic_word, left_on='uid', right_on='用户id', how='left').drop(['uid'], axis=1)
    #topic_id encode一下
    print(user_topic_word.info())
    
    user_topic_word['u_topic_word_count'] = user_topic_word['u_topic_word_count'].fillna(-1)
    user_topic_word['u_topic_word_count'] = user_topic_word['u_topic_word_count'].astype('int')
    user_topic_word = reduce_mem_usage(user_topic_word)
    question_topic = question_topic.drop_duplicates(subset=None, keep='first', inplace=False)
    user_topic_word = user_topic_word.drop_duplicates(subset=None, keep='first', inplace=False)
    print(user_topic_word.info())
    
    with open(etype+'question_topic.pkl', 'wb') as file:
        pickle.dump(question_topic, file)
    with open(etype+'user_topic_word.pkl', 'wb') as file:
        pickle.dump(user_topic_word, file)    
    print("saved pkl ...") 
    gc.collect()
    
    return 0

# for users_topics
def mk(ls_pair):
    dt = {}
    for pair in ls_pair:
        dt[pair[0]] = pair[1]
    return dt

def mk_u_t_dict(ut_test):
    print("start making u_topic_scores...")
    ut_test = ut_test.drop(index=[0], axis=0).reset_index() # 因为第一行全部都是0，所以删掉。。
    df = ut_test[['user_topic_id','u_topic_word_count']]
    ut_test['dict'] = pd.Series(df.to_dict('split')['data'])
    del df, ut_test['index']
    ut_test = ut_test.groupby('用户id')['dict'].agg(list).to_frame().reset_index()
    ut_test['dict'] = ut_test['dict'].map(mk)
    print(ut_test)

    return ut_test

# for qustion_topics
def ab(df):
    return ','.join(df.values)

def mk_q_t(question_topic):
    print("start making q_topics...")
    question_topic = question_topic.groupby(['问题id'])['question_topic_id'].apply(ab).to_frame()
    question_topic = question_topic.reset_index()
    print(question_topic)
    
    return question_topic


# for the whole run
def extract_user_topic_word_count(train_for_save, inv_label, etype):
    q_t_file = etype+'question_topic.pkl'
    u_t_w_file = etype+'user_topic_word.pkl'
    if not (os.path.exists(q_t_file) and os.path.exists(u_t_w_file)):
        creat_2pkl(answer_qestion_info, inv_label, etype)
    
    with open(q_t_file, 'rb') as file:
        question_topic = pickle.load(file)
    with open(u_t_w_file, 'rb') as file:
        user_topic_word = pickle.load(file)
    
    user_topic_word = mk_u_t_dict(user_topic_word)
    question_topic = mk_q_t(question_topic)    
    
    inv_label = pd.merge(inv_label, user_topic_word, left_on='uid', right_on='用户id', how='inner').drop(['用户id'], axis=1)
    inv_label = pd.merge(inv_label, question_topic, left_on='qid', right_on='问题id', how='inner').drop(['问题id'], axis=1)
    
    print(inv_label.info())
    print(inv_label.head(30))
    return inv_label

### 计数topic关联回答的个数-计算topic热度

In [7]:
def get_topic_hot(answer_qestion_info, train_label, etype):
    answer_qestion_info1 = answer_qestion_info['问题绑定话题'].str.split(',', expand=True).stack()
    #print(answer_qestion_info1)
    answer_qestion_info1 = answer_qestion_info1.reset_index(level=1, drop=True).rename('user_topic_id')
    answer_qestion_info = answer_qestion_info.join(answer_qestion_info1)
    #print(answer_qestion_info.head(10))
    del answer_qestion_info1
    answer_qestion_info = reduce_mem_usage(answer_qestion_info)
    #print(answer_qestion_info.head(30))
    
    topic_list = answer_qestion_info[['问题id','user_topic_id']]
    topic_list['topic_count'] = topic_list['user_topic_id'].map(topic_list['user_topic_id'].value_counts().astype(int))
    topic_list = topic_list[['user_topic_id','topic_count']].drop_duplicates(inplace=False)
    del answer_qestion_info
    
    print(topic_list.head(30))
    with open(etype+'topic_count.pkl', 'wb') as file:
        pickle.dump(topic_count, file)
    return topic_list

def merge_que_topic_count(topic_count, etype):
    
    with open(etype+'question_topic.pkl', 'rb') as file:
        question_topic = pickle.load(file)
    print(question_topic.head(5))
    
    question_label = pd.merge(question_topic, topic_count, left_on='question_topic_id', right_on='user_topic_id', how='inner').drop('question_topic_id',axis=1)
    question_label['q_t_a_count_sum'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_count'].agg('sum').astype(int))
    question_label['q_t_a_count_mean'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_count'].agg('mean'))
    question_label['q_t_a_count_max'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_count'].agg('max').astype(int))
    question_label['q_t_a_count_std'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_count'].agg('std'))
    question_label['q_t_a_count_min'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_count'].agg('min').astype(int))
    
    question_label = question_label[['问题id','q_t_a_count_sum','q_t_a_count_mean','q_t_a_count_max','q_t_a_count_min','q_t_a_count_std']].drop_duplicates(inplace=False)
    
    with open(etype+'que_topic_count.pkl', 'wb') as file:
        pickle.dump(question_label, file)
    
    return question_label

In [8]:
train_len = 2593669

In [ ]:
data = pd.read_hdf(os.path.join(features_path,'data_all.h5'),key='data').reset_index()
train_label = data.iloc[:train_len][['uid','qid']].drop_duplicates(inplace=False)
test = data.iloc[train_len:][['uid','qid']].drop_duplicates(inplace=False)

train_start = 3807
train_end = 3860
val_start = train_start + 7
val_end = train_end + 7

train_for_save2 = answer_qestion_info[answer_qestion_info['回答创建时间-day']>=train_start][answer_qestion_info['回答创建时间-day']<=train_end]
test_for_save2 = answer_qestion_info[answer_qestion_info['回答创建时间-day']>=val_start][answer_qestion_info['回答创建时间-day']<=val_end]

extract_user_topic_word_count(train_for_save2, train_label, 'train')
extract_user_topic_word_count(test_for_save2, test, 'val')

train_topic_count = get_topic_hot(train_for_save2, train_label, 'train')
train_que_topic_count = merge_que_topic_count(train_topic_count, 'train')

test_topic_count = get_topic_hot(test_for_save2, test, 'val')
test_que_topic_count = merge_que_topic_count(test_topic_count, 'val')

/home/jiyangguang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()


start making u_topic_scores...


### 计算topic的关联回答的点赞反馈-计算topic的质量

In [ ]:
def get_topic_good(answer_qestion_info, train_label, etype):
    answer_qestion_info1 = answer_qestion_info['问题绑定话题'].str.split(',', expand=True).stack()
    #print(answer_qestion_info1)
    answer_qestion_info1 = answer_qestion_info1.reset_index(level=1, drop=True).rename('user_topic_id')
    answer_qestion_info = answer_qestion_info.join(answer_qestion_info1)
    #print(answer_qestion_info.head(10))
    del answer_qestion_info1
    answer_qestion_info = reduce_mem_usage(answer_qestion_info)
    print(answer_qestion_info.head(30))
    
    topic_list = answer_qestion_info[['问题id','user_topic_id', '点赞数']]
    topic_list['topic_good_count'] = topic_list['user_topic_id'].map(topic_list.groupby('user_topic_id')['点赞数'].agg('sum').astype(int))
    topic_list = topic_list[['user_topic_id','topic_good_count']].drop_duplicates(inplace=False)
    del answer_qestion_info  
    
    print(topic_list.head(30))
    with open(etype+'topic_good_count.pkl', 'wb') as file:
        pickle.dump(topic_count, file)
    return topic_list

def merge_que_topic_good(topic_count, etype):
    
    with open(etype+'question_topic.pkl', 'rb') as file:
        question_topic = pickle.load(file)
    print(question_topic.head(5))
    
    question_label = pd.merge(question_topic, topic_count, left_on='question_topic_id', right_on='user_topic_id', how='inner').drop('question_topic_id',axis=1)
    question_label['q_t_a_good_count_sum'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_good_count'].agg('sum').astype(int))
    question_label['q_t_a_good_count_mean'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_good_count'].agg('mean'))
    question_label['q_t_a_good_count_max'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_good_count'].agg('max').astype(int))
    question_label['q_t_a_good_count_std'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_good_count'].agg('std'))
    question_label['q_t_a_good_count_min'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_good_count'].agg('min').astype(int))
    
    question_label = question_label[['问题id','q_t_a_good_count_sum','q_t_a_good_count_mean','q_t_a_good_count_max','q_t_a_good_count_min','q_t_a_good_count_std']].drop_duplicates(inplace=False)
    
    with open(etype+'que_topic_good_count.pkl', 'wb') as file:
        pickle.dump(question_label, file)
        
    print(question_label.head())
    
    return question_label

In [ ]:
data = pd.read_hdf(os.path.join(features_path,'data_all.h5'),key='data').reset_index()
train_label = data.iloc[:train_len][['uid','qid']].drop_duplicates(inplace=False)
test = data.iloc[train_len:][['uid','qid']].drop_duplicates(inplace=False)

train_start = 3807
train_end = 3860
val_start = train_start + 7
val_end = train_end + 7

train_for_save2 = answer_qestion_info[answer_qestion_info['回答创建时间-day']>=train_start][answer_qestion_info['回答创建时间-day']<=train_end]
test_for_save2 = answer_qestion_info[answer_qestion_info['回答创建时间-day']>=val_start][answer_qestion_info['回答创建时间-day']<=val_end]

train_topic_good_count = get_topic_good(train_for_save2, train_label, 'train')
train_que_topic_good_count = merge_que_topic_good(train_topic_good_count, 'train')

test_topic_good_count = get_topic_good(test_for_save2, test, 'val')
test_que_topic_good_count = merge_que_topic_good(test_topic_good_count, 'val')

print(train_que_topic_good_count.info())
print(test_que_topic_good_count.info())

### 根据topic关联回答的收藏反馈- 计算topic的质量

In [ ]:
def get_topic_mark(answer_qestion_info, train_label, etype):
    answer_qestion_info1 = answer_qestion_info['问题绑定话题'].str.split(',', expand=True).stack()
    #print(answer_qestion_info1)
    answer_qestion_info1 = answer_qestion_info1.reset_index(level=1, drop=True).rename('user_topic_id')
    answer_qestion_info = answer_qestion_info.join(answer_qestion_info1)
    #print(answer_qestion_info.head(10))
    del answer_qestion_info1
    answer_qestion_info = reduce_mem_usage(answer_qestion_info)
    print(answer_qestion_info.head(30))
    
    topic_list = answer_qestion_info[['问题id','user_topic_id', '收藏数']]
    topic_list['topic_mark_count'] = topic_list['user_topic_id'].map(topic_list.groupby('user_topic_id')['收藏数'].agg('sum').astype(int))
    topic_list = topic_list[['user_topic_id','topic_mark_count']].drop_duplicates(inplace=False)
    del answer_qestion_info  
    
    print(topic_list.head(30))
    with open(etype+'topic_mark_count.pkl', 'wb') as file:
        pickle.dump(topic_count, file)
    return topic_list

def merge_que_topic_mark(topic_count, etype):
    
    with open(etype+'question_topic.pkl', 'rb') as file:
        question_topic = pickle.load(file)
    print(question_topic.head(5))
    
    question_label = pd.merge(question_topic, topic_count, left_on='question_topic_id', right_on='user_topic_id', how='inner').drop('question_topic_id',axis=1)
    question_label['q_t_a_mark_count_sum'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_mark_count'].agg('sum').astype(int))
    question_label['q_t_a_mark_count_mean'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_mark_count'].agg('mean'))
    question_label['q_t_a_mark_count_max'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_mark_count'].agg('max').astype(int))
    question_label['q_t_a_mark_count_std'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_mark_count'].agg('std'))
    question_label['q_t_a_mark_count_min'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_mark_count'].agg('min').astype(int))
    
    question_label = question_label[['问题id','q_t_a_mark_count_sum','q_t_a_mark_count_mean','q_t_a_mark_count_max','q_t_a_mark_count_min','q_t_a_mark_count_std']].drop_duplicates(inplace=False)
    
    with open(etype+'que_topic_mark_count.pkl', 'wb') as file:
        pickle.dump(question_label, file)
        
    print(question_label.head())
    
    return question_label

In [ ]:
data = pd.read_hdf(os.path.join(features_path,'data_all.h5'),key='data').reset_index()
train_label = data.iloc[:train_len][['uid','qid']].drop_duplicates(inplace=False)
test = data.iloc[train_len:][['uid','qid']].drop_duplicates(inplace=False)

train_start = 3807
train_end = 3860
val_start = train_start + 7
val_end = train_end + 7

train_for_save2 = answer_qestion_info[answer_qestion_info['回答创建时间-day']>=train_start][answer_qestion_info['回答创建时间-day']<=train_end]
test_for_save2 = answer_qestion_info[answer_qestion_info['回答创建时间-day']>=val_start][answer_qestion_info['回答创建时间-day']<=val_end]

train_topic_mark_count = get_topic_mark(train_for_save2, train_label, 'train')
train_que_topic_mark_count = merge_que_topic_mark(train_topic_mark_count, 'train')

test_topic_mark_count = get_topic_mark(test_for_save2, test, 'val')
test_que_topic_mark_count = merge_que_topic_mark(test_topic_mark_count, 'val')

print(train_que_topic_mark_count.info())
print(test_que_topic_mark_count.info())

### 根据topic关联回答的感谢反馈- 计算topic的质量

In [ ]:
def get_topic_thank(answer_qestion_info, train_label, etype):
    answer_qestion_info1 = answer_qestion_info['问题绑定话题'].str.split(',', expand=True).stack()
    #print(answer_qestion_info1)
    answer_qestion_info1 = answer_qestion_info1.reset_index(level=1, drop=True).rename('user_topic_id')
    answer_qestion_info = answer_qestion_info.join(answer_qestion_info1)
    #print(answer_qestion_info.head(10))
    del answer_qestion_info1
    answer_qestion_info = reduce_mem_usage(answer_qestion_info)
    print(answer_qestion_info.head(30))
    
    topic_list = answer_qestion_info[['问题id','user_topic_id', '感谢数']]
    topic_list['topic_thank_count'] = topic_list['user_topic_id'].map(topic_list.groupby('user_topic_id')['感谢数'].agg('sum').astype(int))
    topic_list = topic_list[['user_topic_id','topic_thank_count']].drop_duplicates(inplace=False)
    del answer_qestion_info  
    
    print(topic_list.head(30))
    with open(etype+'topic_thank_count.pkl', 'wb') as file:
        pickle.dump(topic_count, file)
    return topic_list

def merge_que_topic_thank(topic_count, etype):
    
    with open(etype+'question_topic.pkl', 'rb') as file:
        question_topic = pickle.load(file)
    print(question_topic.head(5))
    
    question_label = pd.merge(question_topic, topic_count, left_on='question_topic_id', right_on='user_topic_id', how='inner').drop('question_topic_id',axis=1)
    question_label['q_t_a_thank_count_sum'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_thank_count'].agg('sum').astype(int))
    question_label['q_t_a_thank_count_mean'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_thank_count'].agg('mean'))
    question_label['q_t_a_thank_count_max'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_thank_count'].agg('max').astype(int))
    question_label['q_t_a_thank_count_std'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_thank_count'].agg('std'))
    question_label['q_t_a_thank_count_min'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_thank_count'].agg('min').astype(int))
    
    question_label = question_label[['问题id','q_t_a_thank_count_sum','q_t_a_thank_count_mean','q_t_a_thank_count_max','q_t_a_thank_count_min','q_t_a_thank_count_std']].drop_duplicates(inplace=False)
    
    with open(etype+'que_topic_thank_count.pkl', 'wb') as file:
        pickle.dump(question_label, file)
        
    print(question_label.head())
    
    return question_label

In [ ]:
data = pd.read_hdf(os.path.join(features_path,'data_all.h5'),key='data').reset_index()
train_label = data.iloc[:train_len][['uid','qid']].drop_duplicates(inplace=False)
test = data.iloc[train_len:][['uid','qid']].drop_duplicates(inplace=False)

train_start = 3807
train_end = 3860
val_start = train_start + 7
val_end = train_end + 7

train_for_save2 = answer_qestion_info[answer_qestion_info['回答创建时间-day']>=train_start][answer_qestion_info['回答创建时间-day']<=train_end]
test_for_save2 = answer_qestion_info[answer_qestion_info['回答创建时间-day']>=val_start][answer_qestion_info['回答创建时间-day']<=val_end]

train_topic_thank_count = get_topic_thank(train_for_save2, train_label, 'train')
train_que_topic_thank_count = merge_que_topic_thank(train_topic_thank_count, 'train')

test_topic_thank_count = get_topic_thank(test_for_save2, test, 'val')
test_que_topic_thank_count = merge_que_topic_thank(test_topic_thank_count, 'val')

print(train_que_topic_thank_count.info())
print(test_que_topic_thank_count.info())

### 计算topic在inv中的接受程度 - 计算topic的转化率

In [ ]:
def get_topic_trans(inv_qestion_info, train_label, etype):
    inv_qestion_info1 = inv_qestion_info['问题绑定话题'].str.split(',', expand=True).stack()
    #print(inv_qestion_info1)
    inv_qestion_info1 = inv_qestion_info1.reset_index(level=1, drop=True).rename('user_inv_topic_id')
    inv_qestion_info = inv_qestion_info.join(inv_qestion_info1)
    #print(inv_qestion_info.head(10))
    del inv_qestion_info1
    inv_qestion_info = reduce_mem_usage(inv_qestion_info)
    print(inv_qestion_info.head(30))
    
    topic_list = inv_qestion_info[['问题id','user_inv_topic_id', '是否回答']]
    topic_list['topic_inv_pos_mean'] = topic_list['user_inv_topic_id'].map(topic_list.groupby('user_inv_topic_id')['是否回答'].agg('mean'))
    topic_list['topic_inv_neg_mean'] = topic_list['user_inv_topic_id'].map(topic_list.groupby('user_inv_topic_id')['是否回答'].agg(lambda x: 1-np.mean(x)))
    topic_list['topic_inv_sum'] = topic_list['user_inv_topic_id'].map(topic_list.groupby('user_inv_topic_id')['是否回答'].agg('sum').astype(int))
    topic_list['topic_inv_std'] = topic_list['user_inv_topic_id'].map(topic_list.groupby('user_inv_topic_id')['是否回答'].agg('std'))
    topic_list['topic_inv_count'] = topic_list['user_inv_topic_id'].map(topic_list['user_inv_topic_id'].value_counts().astype(int))

    topic_list = topic_list[['user_inv_topic_id','topic_inv_pos_mean','topic_inv_neg_mean','topic_inv_sum','topic_inv_std','topic_inv_count']].drop_duplicates(inplace=False)
    del inv_qestion_info  
    
    print(topic_list.head(30))
    with open(etype+'topic_inv_count.pkl', 'wb') as file:
        pickle.dump(topic_count, file)
    return topic_list

def merge_que_topic_trans(topic_count, etype):
    
    with open(etype+'question_topic.pkl', 'rb') as file:
        question_topic = pickle.load(file)
    print(question_topic.head(5))
    
    question_label = pd.merge(question_topic, topic_count, left_on='question_topic_id', right_on='user_inv_topic_id', how='inner').drop('question_topic_id',axis=1)
    #question_label = topic_count
    question_label['q_t_a_inv_mean_sum'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_inv_pos_mean'].agg('sum').astype(int))
    question_label['q_t_a_inv_mean_mean'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_inv_pos_mean'].agg('mean'))
    question_label['q_t_a_inv_mean_max'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_inv_pos_mean'].agg('max').astype(int))
    question_label['q_t_a_inv_mean_std'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_inv_pos_mean'].agg('std'))
    question_label['q_t_a_inv_mean_min'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_inv_pos_mean'].agg('min').astype(int))
    
    question_label['q_t_a_inv_std_mean'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_inv_std'].agg('mean'))
    question_label['q_t_a_inv_sum_mean'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_inv_sum'].agg('mean'))
    question_label['q_t_a_inv_count_mean'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_inv_count'].agg('mean'))

    question_label['q_t_a_neg_inv_mean_sum'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_inv_neg_mean'].agg('sum').astype(int))
    question_label['q_t_a_neg_inv_mean_mean'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_inv_neg_mean'].agg('mean'))
    question_label['q_t_a_neg_inv_mean_max'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_inv_neg_mean'].agg('max').astype(int))
    question_label['q_t_a_neg_inv_mean_std'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_inv_neg_mean'].agg('std'))
    question_label['q_t_a_neg_inv_mean_min'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_inv_neg_mean'].agg('min').astype(int))
    
  
    
    question_label = question_label[['问题id','q_t_a_inv_mean_sum','q_t_a_inv_mean_mean','q_t_a_inv_mean_max','q_t_a_inv_mean_min','q_t_a_inv_mean_std',
                                    'q_t_a_inv_std_mean', 'q_t_a_inv_sum_mean', 'q_t_a_inv_count_mean', 
                                     'q_t_a_neg_inv_mean_sum', 'q_t_a_neg_inv_mean_mean', 'q_t_a_neg_inv_mean_max', 'q_t_a_neg_inv_mean_std', 'q_t_a_neg_inv_mean_min'
                                    ]].drop_duplicates(inplace=False)
    
    with open(etype+'que_topic_inv_count.pkl', 'wb') as file:
        pickle.dump(question_label, file)
        
    print(question_label.head())
    
    return question_label

In [ ]:
print(train_label.info())

In [ ]:
inv_qestion_info = pd.read_csv(os.path.join(data_path, 'invite_info_0926.txt'), header=None, sep='\t')
inv_qestion_info.columns = ['问题id', '用户id', '邀请创建时间','是否回答']
inv_qestion_info['邀请创建时间-day'] = inv_qestion_info['邀请创建时间'].apply(lambda x:x.split('-')[0].split('D')[1]).astype(int)
del inv_qestion_info['邀请创建时间'] 

data = pd.read_hdf(os.path.join(features_path,'data_all.h5'),key='data').reset_index()
train_label = data.iloc[:train_len][['uid','qid','label','day']].drop_duplicates(inplace=False)
train_label = pd.merge(train_label, question_info, left_on='qid', right_on='问题id', how='left').drop(['问题id'], axis=1, inplace=False)

test = data.iloc[train_len:][['uid','qid','label','day']].drop_duplicates(inplace=False)
test = pd.merge(test, question_info, left_on='qid', right_on='问题id', how='left').drop(['问题id'], axis=1, inplace=False)

train_start = 3837
train_end = 3860
val_start = train_start + 7
val_end = train_end + 7

train_for_save3 = inv_qestion_info[inv_qestion_info['邀请创建时间-day']>=train_start][inv_qestion_info['邀请创建时间-day']<=train_end]
test_for_save3 = inv_qestion_info[inv_qestion_info['邀请创建时间-day']>=val_start][inv_qestion_info['邀请创建时间-day']<=val_end]
train_for_save3 = pd.merge(train_for_save3, question_info, on='问题id', how='left')
test_for_save3 = pd.merge(test_for_save3, question_info, on='问题id', how='left')

In [ ]:
print(train_for_save3.info())

In [ ]:
train_topic_trans = get_topic_trans(train_for_save3, train_label, 'train')
train_que_topic_trans = merge_que_topic_trans(train_topic_trans, 'train')

test_topic_trans = get_topic_trans(test_for_save3, test, 'val')
test_que_topic_trans = merge_que_topic_trans(test_topic_trans, 'val')

### 总结汇总到一个表里

In [275]:
train_q_a_topic = pd.merge(train_que_topic_count, train_que_topic_good_count, on='问题id', how='outer')
test_q_a_topic = pd.merge(test_que_topic_count, test_que_topic_good_count, on='问题id', how='outer')
train_q_a_topic = pd.merge(train_q_a_topic, train_que_topic_trans, on='问题id', how='outer')
test_q_a_topic = pd.merge(test_q_a_topic, test_que_topic_trans, on='问题id', how='outer')

In [276]:
train_q_a_topic = pd.merge(train_q_a_topic, train_que_topic_mark_count, on='问题id', how='outer')
test_q_a_topic = pd.merge(test_q_a_topic, train_que_topic_mark_count, on='问题id', how='outer')

train_q_a_topic = pd.merge(train_q_a_topic, train_que_topic_thank_count, on='问题id', how='outer')
test_q_a_topic = pd.merge(test_q_a_topic, train_que_topic_thank_count, on='问题id', how='outer')

print(train_q_a_topic.info())
print(test_q_a_topic.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1828312 entries, 0 to 1828311
Data columns (total 34 columns):
问题id                       object
q_t_a_count_sum            float64
q_t_a_count_mean           float64
q_t_a_count_max            float64
q_t_a_count_min            float64
q_t_a_count_std            float64
q_t_a_good_count_sum       float64
q_t_a_good_count_mean      float64
q_t_a_good_count_max       float64
q_t_a_good_count_min       float64
q_t_a_good_count_std       float64
q_t_a_inv_mean_sum         float64
q_t_a_inv_mean_mean        float64
q_t_a_inv_mean_max         float64
q_t_a_inv_mean_min         float64
q_t_a_inv_mean_std         float64
q_t_a_inv_std_mean         float64
q_t_a_inv_sum_mean         float64
q_t_a_inv_count_mean       float64
q_t_a_neg_inv_mean_sum     float64
q_t_a_neg_inv_mean_mean    float64
q_t_a_neg_inv_mean_max     float64
q_t_a_neg_inv_mean_std     float64
q_t_a_neg_inv_mean_min     float64
q_t_a_mark_count_sum       float64
q_t_a_mark_co

In [277]:
with open('que_topic_all_count_train.pkl', 'wb') as file:
    pickle.dump(train_q_a_topic, file)
with open('que_topic_all_count_val.pkl', 'wb') as file:
    pickle.dump(test_q_a_topic, file)

### ------------分割线------------------

In [330]:
data = pd.read_hdf(os.path.join(features_path,'data_all.h5'),key='data').reset_index()
train_label = data.iloc[:train_len][['uid','qid']].drop_duplicates(inplace=False)
test = data.iloc[train_len:][['uid','qid']].drop_duplicates(inplace=False)

train_start = 3807
train_end = 3860
val_start = train_start + 7
val_end = train_end + 7

train_for_save = answer_qestion_info[answer_qestion_info['回答创建时间-day']>=train_start][answer_qestion_info['回答创建时间-day']<=train_end]
test_for_save = answer_qestion_info[answer_qestion_info['回答创建时间-day']>=val_start][answer_qestion_info['回答创建时间-day']<=val_end]

/home/jiyangguang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()


In [333]:
print(train_for_save.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3828707 entries, 0 to 4513734
Data columns (total 9 columns):
回答id          object
用户id          object
问题id          object
问题绑定话题        object
回答字数          int64
点赞数           int64
感谢数           int64
收藏数           int64
回答创建时间-day    int64
dtypes: int64(5), object(4)
memory usage: 292.1+ MB
None


In [334]:
print(train_label.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2593066 entries, 0 to 2593668
Data columns (total 2 columns):
uid    object
qid    object
dtypes: object(2)
memory usage: 59.4+ MB
None


In [318]:
train_for_save1 = extract_user_topic_word_count(train_for_save, train_label, 'train')
test_for_save1 = extract_user_topic_word_count(test_for_save, test, 'val')

start making u_topic_scores...
               用户id                                               dict
0       M1000000382                              {'T13': 0, 'T567': 0}
1       M1000000983  {'T6192': 8, 'T1667': 56, 'T2255': 31, 'T1103'...
2       M1000008978  {'T1891': 6, 'T56': 6, 'T5': 6, 'T39': 6, 'T13...
3       M1000020034                                       {'T597': 90}
4       M1000025214                           {'T86': 226, 'T74': 226}
5       M1000031027  {'T49': 24, 'T82': 24, 'T72': 30, 'T42': 30, '...
6       M1000043012  {'T4557': 710, 'T543': 506, 'T1655': 277, 'T55...
7       M1000047678  {'T7': 10, 'T2': 10, 'T35': 10, 'T252': 10, 'T...
8       M1000052881  {'T1201': 95, 'T10855': 95, 'T11899': 95, 'T41...
9       M1000054663  {'T539': 858, 'T132': 180, 'T272': 439, 'T491'...
10      M1000055649                           {'T402': 56, 'T849': 56}
11      M1000058328        {'T234': 7, 'T101': 7, 'T2': 7, 'T5807': 7}
12      M1000063561  {'T37': 920, 'T79364': 29

In [319]:
# 处理dict列的空值
train_for_save1 = train_for_save1[train_for_save1['dict']!=-1].reset_index().drop('index', axis=1)
test_for_save1 = test_for_save1[test_for_save1['dict']!=-1].reset_index().drop('index', axis=1)

In [320]:
del train_for_save1['qid'], train_for_save1['question_topic_id']
del test_for_save1['qid'], test_for_save1['question_topic_id']

In [329]:
with open('u_topic_a_word_count_dict_train.pkl', 'wb') as file:
    pickle.dump(train_for_save1, file)
with open('u_topic_a_word_count_dict_val.pkl', 'wb') as file:
    pickle.dump(test_for_save1, file)

### 计算用户累计回答字数最多的话题

In [337]:
train_user_topic_zishu = train_for_save1.drop_duplicates(subset=['uid'], keep='first', inplace=False).reset_index().drop('index', axis=1)
test_user_topic_zishu = test_for_save1.drop_duplicates(subset=['uid'], keep='first', inplace=False).reset_index().drop('index', axis=1)
print(train_user_topic_zishu.info())
print(test_user_topic_zishu.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 618787 entries, 0 to 618786
Data columns (total 2 columns):
uid     618787 non-null object
dict    618787 non-null object
dtypes: object(2)
memory usage: 9.4+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424048 entries, 0 to 424047
Data columns (total 2 columns):
uid     424048 non-null object
dict    424048 non-null object
dtypes: object(2)
memory usage: 6.5+ MB
None


In [338]:
#需要对topic id进行统一编码
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
topic = pd.read_csv(os.path.join(data_path, 'topic_vectors_64d.txt'), 
                          names=['id', 'embed'], sep='\t')
topic_list = topic['id']
topic_list = topic_list.append(pd.Series(['-1']))
encoder.fit(topic_list)
#data[feat] = encoder.transform(data[feat])

LabelEncoder()

In [339]:
def most_wordcount_topic(d):
    if len(d) == 0:
        return '-1'
    return list(d.keys())[np.argmax(list(d.values()))]

def get_interest_values(d):
    if len(d) == 0:
        return [0]
    return list(d.values())

def get_answer_topics(d):
    if len(d) == 0:
        return '-1'
    return list(d.keys())

def get_user_topic_zishu(train_user_topic_zishu):
    #train_user_topic_zishu['u_t_most_countword_id'] = train_user_topic_zishu['dict'].apply(most_wordcount_topic)
    
    #train_user_topic_zishu['u_t_most_countword_id'] = encoder.transform(train_user_topic_zishu['u_t_most_countword_id'])
    
    train_user_topic_zishu['u_t_a_count'] = train_user_topic_zishu['dict'].apply(len)

    train_user_topic_zishu['u_t_countword_values'] = train_user_topic_zishu['dict'].apply(get_interest_values)
    #train_user_topic_zishu['u_t_min_countword_values'] = train_user_topic_zishu['u_t_countword_values'].apply(np.min)
    train_user_topic_zishu['u_t_max_countword_values'] = train_user_topic_zishu['u_t_countword_values'].apply(np.max)
    #train_user_topic_zishu['u_t_mean_countword_values'] = train_user_topic_zishu['u_t_countword_values'].apply(np.mean)
    train_user_topic_zishu['u_t_std_countword_values'] = train_user_topic_zishu['u_t_countword_values'].apply(np.std)
    
    del train_user_topic_zishu['dict']#, train_user_topic_zishu['u_t_countword_values']

    return train_user_topic_zishu

def get_user_topic_answered(train_user_topic_zishu):
    #train_user_topic_zishu['u_t_most_countword_id'] = train_user_topic_zishu['dict'].apply(most_wordcount_topic)
    
    #train_user_topic_zishu['u_t_most_countword_id'] = encoder.transform(train_user_topic_zishu['u_t_most_countword_id'])
    
    #train_user_topic_zishu['u_t_a_count'] = train_user_topic_zishu['dict'].apply(len)

    train_user_topic_zishu['u_t_answered_topics'] = train_user_topic_zishu['dict'].apply(get_answer_topics)
    #train_user_topic_zishu['u_t_min_countword_values'] = train_user_topic_zishu['u_t_countword_values'].apply(np.min)
    #train_user_topic_zishu['u_t_max_countword_values'] = train_user_topic_zishu['u_t_countword_values'].apply(np.max)
    #train_user_topic_zishu['u_t_mean_countword_values'] = train_user_topic_zishu['u_t_countword_values'].apply(np.mean)
    #train_user_topic_zishu['u_t_std_countword_values'] = train_user_topic_zishu['u_t_countword_values'].apply(np.std)
    
    del train_user_topic_zishu['dict']#, train_user_topic_zishu['u_t_countword_values']

    return train_user_topic_zishu

In [326]:
train_user_topic_zishu1 = get_user_topic_zishu(train_user_topic_zishu)
test_user_topic_zishu1 = get_user_topic_zishu(test_user_topic_zishu)

In [340]:
train_user_topic_answered = get_user_topic_answered(train_user_topic_zishu)
test_user_topic_answered = get_user_topic_answered(test_user_topic_zishu)

In [341]:
print(train_user_topic_answered.head())
print(test_user_topic_answered.head())

           uid                                u_t_answered_topics
0  M2317670257  [T10, T10186, T42650, T184, T1203, T4454, T564...
1   M893603481  [T4385, T2863, T70, T9160, T1018, T1749, T97, ...
2  M1699039805  [T2, T47, T22, T3, T950, T566, T52, T1389, T65...
3  M3147978248                     [T208, T5339, T927, T271, T11]
4   M866789582                            [T32, T81, T524, T2522]
           uid                                u_t_answered_topics
0    M64135255                                       [T93, T3887]
1  M4033621585  [T808, T1751, T42, T456, T241, T534, T86, T111...
2  M1582438278                            [T660, T101, T234, T93]
3  M1225946888  [T674, T2, T22, T36, T31, T492, T224, T45384, ...
4  M4063235921                  [T11431, T101, T1469, T234, T150]


In [354]:
def get_user_sparse_atopic(answer_qestion_info, etype):
    answer_qestion_info1 = answer_qestion_info.reindex(answer_qestion_info.index.repeat(answer_qestion_info.u_t_answered_topics.str.len())).assign(u_t_answered_topics=np.concatenate(answer_qestion_info.u_t_answered_topics.values))
    #answer_qestion_info1 = answer_qestion_info['u_t_answered_topics'].str.split(',', expand=True)#.stack()
    print(answer_qestion_info1)
    answer_qestion_info1 = answer_qestion_info1.rename({' u_t_answered_topics':'u_t_answered_single_topic'})
    #answer_qestion_info = answer_qestion_info.join(answer_qestion_info1)
    #print(answer_qestion_info.head(10))
    del answer_qestion_info
    answer_qestion_info1 = reduce_mem_usage(answer_qestion_info1)
    print(answer_qestion_info1.head(10))
    
    with open(etype+'user_topic.pkl', 'wb') as file:
        pickle.dump(answer_qestion_info1, file)   
        
    return answer_qestion_info1

train_user_topic_answered1 = get_user_sparse_atopic(train_user_topic_answered, 'train')
test_user_topic_answered1 = get_user_sparse_atopic(test_user_topic_answered, 'val')

                uid u_t_answered_topics
0       M2317670257                 T10
0       M2317670257              T10186
0       M2317670257              T42650
0       M2317670257                T184
0       M2317670257               T1203
0       M2317670257               T4454
0       M2317670257               T5649
0       M2317670257               T5122
1        M893603481               T4385
1        M893603481               T2863
1        M893603481                 T70
1        M893603481               T9160
1        M893603481               T1018
1        M893603481               T1749
1        M893603481                 T97
1        M893603481                T100
1        M893603481               T1813
1        M893603481                T407
1        M893603481               T1262
1        M893603481               T7007
1        M893603481              T41931
1        M893603481               T3460
1        M893603481              T59710
1        M893603481               T7019


In [ ]:
def get_topic_hot_for_user(answer_qestion_info, train_label, etype):
    topic_list = answer_qestion_info[['uid','u_t_answered_topics']]
    topic_list['topic_count'] = topic_list['user_topic_id'].map(topic_list['user_topic_id'].value_counts().astype(int))
    topic_list = topic_list[['user_topic_id','topic_count']].drop_duplicates(inplace=False)
    del answer_qestion_info
    
    print(topic_list.head(30))
    with open(etype+'topic_count.pkl', 'wb') as file:
        pickle.dump(topic_count, file)
    return topic_list

def merge_que_topic_count(topic_count, etype):
    
    with open(etype+'question_topic.pkl', 'rb') as file:
        question_topic = pickle.load(file)
    print(question_topic.head(5))
    
    question_label = pd.merge(question_topic, topic_count, left_on='question_topic_id', right_on='user_topic_id', how='inner').drop('question_topic_id',axis=1)
    question_label['q_t_a_count_sum'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_count'].agg('sum').astype(int))
    question_label['q_t_a_count_mean'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_count'].agg('mean'))
    question_label['q_t_a_count_max'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_count'].agg('max').astype(int))
    question_label['q_t_a_count_std'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_count'].agg('std'))
    question_label['q_t_a_count_min'] = question_label['问题id'].map(question_label.groupby('问题id')['topic_count'].agg('min').astype(int))
    
    question_label = question_label[['问题id','q_t_a_count_sum','q_t_a_count_mean','q_t_a_count_max','q_t_a_count_min','q_t_a_count_std']].drop_duplicates(inplace=False)
    
    with open(etype+'que_topic_count.pkl', 'wb') as file:
        pickle.dump(question_label, file)
    
    return question_label

In [ ]:
train_len = 2593669

In [ ]:
with open('u_topic_answered_train.pkl', 'wb') as file:
    pickle.dump(train_user_topic_zishu1, file)
with open('u_topic_answered_val.pkl', 'wb') as file:
    pickle.dump(test_user_topic_zishu1, file)

In [314]:
with open('u_topic_a_zishu_train.pkl', 'wb') as file:
    pickle.dump(train_user_topic_zishu1, file)
with open('u_topic_a_zishu_val.pkl', 'wb') as file:
    pickle.dump(test_user_topic_zishu1, file)

### 计算用户累计得到最多赞的话题

In [125]:
import multiprocessing as mp
import gc

# 分割 df，方便多进程跑
def split_df(df, n):
    chunk_size = int(np.ceil(len(df) / n))
    return [df[i*chunk_size:(i+1)*chunk_size] for i in range(n)]

def gc_mp(pool, ret, chunk_list):
    del pool
    for r in ret:
        del r
    del ret
    for cl in chunk_list:
        del cl
    del chunk_list
    gc.collect()
    
# 用户感兴趣topic和问题 topic的交集
def process(df):
    return df.apply(lambda row: list(set(row['dict'].keys()) & set(row['question_topic_id'])),axis=1)

pool = mp.Pool()
chunk_list = split_df(train_for_save1, 100)
ret = pool.map(process, chunk_list)
train_for_save1['topic_answer_intersection'] = pd.concat(ret)
gc_mp(pool, ret, chunk_list)

# 用户感兴趣topic和问题 topic的交集的兴趣值
def process(df):
    return df.apply(lambda row: [row['dict'][t] for t in row['topic_answer_intersection']],axis=1)

pool = mp.Pool()
chunk_list = split_df(train_for_save1, 100)
ret = pool.map(process, chunk_list)
train_for_save1['topic_answer_intersection_values'] = pd.concat(ret)
gc_mp(pool, ret, chunk_list)

# 交集topic计数
train_for_save1['num_topic_answer_intersection'] = train_for_save1['topic_answer_intersection'].apply(len)

Process ForkPoolWorker-351:
Process ForkPoolWorker-370:
Process ForkPoolWorker-350:
Process ForkPoolWorker-358:
Process ForkPoolWorker-363:
Process ForkPoolWorker-346:
Process ForkPoolWorker-344:
  File "/home/jiyangguang/anaconda3/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
Process ForkPoolWorker-354:
Process ForkPoolWorker-362:
Process ForkPoolWorker-360:
Process ForkPoolWorker-365:
Process ForkPoolWorker-347:
Process ForkPoolWorker-364:
Process ForkPoolWorker-345:
Process ForkPoolWorker-357:
Process ForkPoolWorker-367:
Process ForkPoolWorker-348:
Process ForkPoolWorker-349:
Process ForkPoolWorker-353:
Process ForkPoolWorker-356:
Process ForkPoolWorker-343:
Process ForkPoolWorker-332:
Process ForkPoolWorker-337:
Process ForkPoolWorker-361:
Process ForkPoolWorker-339:
Process ForkPoolWorker-340:
Process ForkPoolWorker-342:
Process ForkPoolWorker-366:
Process ForkPoolWorker-359:
Process ForkPoolWorker-352:
Process ForkPoolWorker-335:
Process ForkPoo

In [128]:
print(train_for_save1[train_for_save1['num_topic_answer_intersection']!=0])

Empty DataFrame
Columns: [uid, qid, dict, question_topic_id, topic_answer_intersection, topic_answer_intersection_values, num_topic_answer_intersection]
Index: []
